In [1]:
%matplotlib inline
import os
import importlib
import yaml

import xarray as xr
import numpy as np

import intake
import intake_esm
import esmlab
import xcollection as xc

import pop_tools  # TODO: develop into new package
import esmlab_stats_dev  # TODO: push this into esmlab

import config

In [2]:
USER = os.environ['USER']
import dask
from dask.distributed import Client
from dask_jobqueue import PBSCluster

Nnodes = 4
processes = 18
project = 'NCGD0011'

cluster = PBSCluster(queue='regular',
                     cores = 18,
                     processes = processes,
                     memory = '100GB',          
                     project = project,
                     walltime = '04:00:00',
                     local_directory=f'/glade/scratch/{USER}/dask-tmp')
client = Client(cluster)

cluster.scale(processes*Nnodes)

/glade/work/mclong/miniconda3/envs/notebook-gallery/lib/python3.6/site-packages/distributed/bokeh/core.py:57: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


In [3]:
intake_esm.get_options()

{'database_directory': '/glade/work/mclong/intake-collections',
 'cache_directory': '/glade/scratch/mclong/intake-cesm-data'}

In [4]:
xc.get_options()

{'cache_directory': '/glade/scratch/mclong/notebook-gallery-data'}

In [5]:
col = intake.open_cesm_metadatastore('cesm1_le')
col.df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 262092 entries, 0 to 262091
Data columns (total 18 columns):
resource            262092 non-null object
resource_type       262092 non-null object
direct_access       262092 non-null bool
experiment          262092 non-null object
case                262092 non-null object
component           262092 non-null object
stream              262092 non-null object
variable            262092 non-null object
date_range          262092 non-null object
ensemble            262092 non-null int64
files               262092 non-null object
files_basename      262092 non-null object
files_dirname       262092 non-null object
ctrl_branch_year    0 non-null float64
year_offset         34112 non-null float64
sequence_order      262092 non-null int64
has_ocean_bgc       262092 non-null bool
grid                52764 non-null object
dtypes: bool(2), float64(2), int64(2), object(12)
memory usage: 34.5+ MB


In [6]:
experiments = ['20C', 'RCP85']
ensembles = col.search(experiment=experiments, has_ocean_bgc=True).results.ensemble.unique().tolist()

In [7]:
query_base = dict(experiment=experiments, ensemble=ensembles, stream='pop.h')      
resource_constraints = dict(resource='GLADE:posix:/glade/collections/cdg/data/cesmLE/CESM-CAM5-BGC-LE')


In [8]:
recipes = 'analysis_recipes.yml'
with open('analysis_recipes.yml') as fid:
    recipes = yaml.load(fid)

## compute global means

In [9]:
%%time
analysis_list = ['global_avg']

variable_list = ['O2']

for analysis in analysis_list:
    # TODO: implement hash-namer inside xcollection
    namer = lambda q: '.'.join(['cesm1_le', '+'.join(q['experiment']), q['variable'], analysis])
    for v in variable_list:
        query = {**dict(variable=v), **query_base, **resource_constraints}
        dc = xc.analyzed_collection(collection='cesm1_le',
                                    analysis_recipe=recipes[analysis],
                                    analysis_name=namer(query),
                                    overwrite_existing=False,
                                    file_format="nc",
                                    **query)


CPU times: user 37min 54s, sys: 1min 47s, total: 39min 42s
Wall time: 49min 59s


In [10]:
%load_ext watermark

In [11]:
%watermark --iversion -g -h -m -v -u -d

xarray      0.11.3
intake      0.4.1
esmlab      v2019.2.1+5.g538185f
xcollection v0.10+16.gcfaf6ad
intake_esm  v2019.2.0+87.gbadcef2
numpy       1.16.1
dask        1.1.1
yaml        3.13
last updated: 2019-02-15 

CPython 3.6.7
IPython 7.1.1

compiler   : GCC 7.3.0
system     : Linux
release    : 3.12.62-60.64.8-default
machine    : x86_64
processor  : x86_64
CPU cores  : 72
interpreter: 64bit
host name  : r6i6n30
Git hash   : 6022ab65cb887e512b489ab2d05bf6b979a0a985
